# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [1]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1, notes="AAPL buy order")
        order2 = self.buy("GOOG", quantity2, notes="GOOG buy order")

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900314  89.141426  130.451989  89.429268  127.713306   
2022-12-27  128.097488  87.275421  129.456984  88.833733  126.806960   
2022-12-28  124.166794  85.816368  129.082630  87.861030  123.999322   
2022-12-29  127.683701  88.287827  128.540766  88.699738  125.831644   
2022-12-30  127.998978  88.069466  128.018685  88.168720  125.536140   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967734  128.974282  86.967734  63814900  17815000  
2022-12-27  86.883365  129.427431  88.645145  69007800  15470900  
2022-12-28  85.727042  127.742842  86.848627  85438400  17879600  
2022-12-29  86.342418  126.087775  86.382121  75703700  18280700  
2022-12-30  86.382116  126.501579  86.714622  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.915344,99983.084656,0,100000.000000
2018-01-03,16.915344,100795.030289,0,100811.945633
2018-01-04,16.915344,101211.215149,0,101228.130493
2018-01-05,16.915344,102526.020256,0,102542.935600
2018-01-08,16.915344,102558.531456,0,102575.446800


## <b> Backtest Statistics </b>

In [5]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.92
Final equity: 241921.36
---------------------------


In [6]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.3631591797,
 'median_profit': 141921.3631591797,
 'mean_profit_win': 141921.3631591797,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.3631591797,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [7]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.3631591797,
 'mean_profit': 112.81507405340197,
 'median_profit': 256.2043228149414,
 'std_profit': 4002.7523033478333,
 'mad_profit': 2424.4152333260404,
 'skewness': -0.1935027177148755,
 'kurtosis': 3.056899838350687,
 'mean_median_gap': -143.38924876153942,
 'mean_profit_over_vol': 0.028184375525571587,
 'median_profit_over_vol': 0.06400703900679952,
 'max_drawdown': -122839.0516204834}

In [8]:
bt.get_equity_statistics_return()

{'total_return': 1.4192136315917967,
 'mean_return': 0.0008838017415807609,
 'median_return': 0.001663444977245132,
 'std_return': 0.019027776608430177,
 'mad_return': 0.01407483347458939,
 'skewness': -0.15457437816137526,
 'kurtosis': 4.071151314796654,
 'mean_median_gap': -0.0007796432356643712,
 'mean_return_over_vol': 0.046447977594460316,
 'median_return_over_vol': 0.08742193118391718,
 'max_drawdown': -0.34324002376637763}

In [9]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,AGE,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE
0,6b392ad5-cb23-4f31-93b7-5b140d9a3c0e,OPEN,Buy and Hold Trade,141921.363159,2018-01-02,None,1823 days,None,"{'AAPL': {'quantity': 1239, 'value': 158590.73...",241904.447815,{},0,None,None,None


In [10]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,NOTES,TRADE_ID,ITERATION_REQUESTED,QUANTITY_REQUESTED,PRICE_REQUESTED,FEE_REQUESTED,GROSS_TOTAL_REQUESTED,TOTAL_REQUESTED,ITERATION_FILLED,QUANTITY_FILLED,PRICE_FILLED,FEE_FILLED,GROSS_TOTAL_FILLED,TOTAL_FILLED
0,159be84e-3b54-448c-8f57-87ed203d4cb6,AAPL,BUY,FILLED,AAPL buy order,6b392ad5-cb23-4f31-93b7-5b140d9a3c0e,2018-01-02,1239,40.341881,0,-49983.590309,-49983.590309,2018-01-02,1239,40.341881,0.0,-49983.590309,-49983.590309
1,caadbf98-ad34-4a2c-95ae-8a1d7d761e5b,GOOG,BUY,FILLED,GOOG buy order,6b392ad5-cb23-4f31-93b7-5b140d9a3c0e,2018-01-02,946,52.853588,0,-49999.494347,-49999.494347,2018-01-02,946,52.853588,0.0,-49999.494347,-49999.494347


In [11]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.73332214355)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.71449279785)}}

In [12]:
bt.strategy.positions_unfilled

{}